In [44]:
!pip install geocoder
!pip install folium

In [45]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium

print("All Libraries imported.")

All Libraries imported.


## Data scraping 

In [46]:
Tor_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup_data = BeautifulSoup(Tor_data, 'html.parser')

soup_data.find('table').find_all('tr')

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>, <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>, <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>, <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>, <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>, <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>, <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>, <tr>
 <td>M2B
 </td>
 <td>Not assigned
 </td>
 <td>Not a

In [47]:
neighborhoodList = []
postalCodeList = []
boroughList = []

for row in soup_data.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [48]:
df_NY = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

df_NY.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [49]:
df_NY_dropna = df_NY[df_NY.Borough != "Not assigned"].reset_index(drop=True)
df_NY_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [50]:
df_NY_grouped = df_NY_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_NY_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [51]:
#Making not assigned Neighborhood equal to their respective borough
for index, row in df_NY_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df_NY_grouped.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [52]:
# creating a new dataframe to store Latitude and longitude data of boroughs

column_names = ["PostalCode", "Borough", "Neighborhood"]
df = pd.DataFrame(columns=column_names)

Newlist = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in Newlist:
    df = df.append(df_NY_grouped[df_NY_grouped["PostalCode"]==postcode], ignore_index=True)
    
df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [53]:
# Getting latitudes data
Geospatial_Data='http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(Geospatial_Data)
df_coordinates.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [54]:
df_coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [55]:
# merging the two tables on the basis of column "PostalCode"
df_NY_new = df_NY_grouped.merge(df_coordinates, on="PostalCode", how="left")
df_NY_new.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [56]:
#Using geopy library to get the latitude and longitude of Toronto

address = 'North York'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


## NY Map

In [57]:

map_NY_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# adding markers to map
for lat, lng, borough, neighborhood in zip(df_NY_new['Latitude'], df_NY_new['Longitude'], df_NY_new['Borough'], df_NY_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='Blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_NY_toronto)  
    
map_NY_toronto

In [58]:
borough_names = list(df_NY_new.Borough.unique())

toronto_borough = []

for x in borough_names:
   
        toronto_borough.append(x)
        
toronto_borough

['Scarborough',
 'North York',
 'East York',
 'East Toronto',
 'Central Toronto',
 'Downtown Toronto',
 'York',
 'West Toronto',
 'Mississauga',
 'Etobicoke']

In [59]:
# defining Foursquare Credentials and Version
CLIENT_ID = 'R5EDE1D5T0UNR435CC3TFUA1ECQQFYA2M0YCDDLA4HDTOXBP'
CLIENT_SECRET = 'ZFN4LZII0LZLFQACOGOQXDL5WXPZCKUKHRQQMGJU5UVRCMJG' 
VERSION = '20200811' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R5EDE1D5T0UNR435CC3TFUA1ECQQFYA2M0YCDDLA4HDTOXBP
CLIENT_SECRET:ZFN4LZII0LZLFQACOGOQXDL5WXPZCKUKHRQQMGJU5UVRCMJG


In [60]:
# Getting the top 100 venues that are within a radius of 500 meters

radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df_NY_new['Latitude'], df_NY_new['Longitude'], df_NY_new['PostalCode'], df_NY_new['Borough'], 
                                                  df_NY_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [61]:
# convert the venues list into a new DataFrame
df_venues = pd.DataFrame(venues)

# define the column names
df_venues.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

df_venues.head()

,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [62]:
#Filtering only North York to a df

df_NY_venues= df_venues[df_venues.Borough == "North York"].reset_index(drop=True)

df_NY_venues.head()

,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,M2H,North York,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,M2H,North York,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,M2H,North York,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,M2H,North York,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


In [63]:
df_NY_venues.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M2H,North York,Hillcrest Village,5,5,5,5,5,5
M2J,North York,"Fairview, Henry Farm, Oriole",67,67,67,67,67,67
M2K,North York,Bayview Village,4,4,4,4,4,4
M2M,North York,"Willowdale, Newtonbrook",1,1,1,1,1,1
M2N,North York,"Willowdale, Willowdale East",34,34,34,34,34,34
M2P,North York,York Mills West,3,3,3,3,3,3
M2R,North York,"Willowdale, Willowdale West",7,7,7,7,7,7
M3A,North York,Parkwoods,4,4,4,4,4,4
M3B,North York,Don Mills,4,4,4,4,4,4


In [64]:
df_NY_venues.shape

(249, 9)

### One hot encoding and analyzing the venues

In [65]:
# Analyzing all the areas

# one hot encoding
NY_onehot = pd.get_dummies(df_NY_venues[['VenueCategory']], prefix="", prefix_sep="")

# adding postal,neighborhood columns back to dataframe
NY_onehot['PostalCode'] = df_NY_venues['PostalCode'] 
NY_onehot['Neighborhoods'] = df_NY_venues['Neighborhood'] 

# moving postal, borough and neighborhood columns to the first column
fixed_columns = list(NY_onehot.columns[-3:]) + list(NY_onehot.columns[:-3])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head(10)

,Women's Store,PostalCode,Neighborhoods,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Locksmith,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant
0,0,M2H,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,M2H,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,M2H,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,M2H,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,M2H,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,M2J,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,0,M2J,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,M2J,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,0,M2J,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,M2J,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [66]:
#grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category
NY_grouped = NY_onehot.groupby(["PostalCode",  "Neighborhoods"]).mean().reset_index()

NY_grouped.head(10)

,PostalCode,Neighborhoods,Women's Store,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Locksmith,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant
0,M2H,Hillcrest Village,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.2,0.000000,0.0,0.0,0.200000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.2,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.20,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,M2J,"Fairview, Henry Farm, Oriole",0.029851,0.014925,0.0,0.014925,0.00,0.000000,0.014925,0.0,0.00,0.029851,0.029851,0.014925,0.014925,0.0,0.00,0.014925,0.0,0.0,0.000000,0.014925,0.014925,0.014925,0.000000,0.000000,0.00,0.0,0.014925,0.029851,0.104478,0.074627,0.0,0.0,0.014925,0.014925,0.0,0.014925,0.00,0.0,0.000000,0.0,0.044776,0.0,0.0,0.059701,0.00,0.014925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014925,0.000000,0.00,0.0,0.014925,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.029851,0.014925,0.029851,0.014925,0.0,0.000000,0.014925,0.0,0.0,0.000000,0.014925,0.0,0.014925,0.0,0.000000,0.000000,0.014925,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.044776,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.000000,0.00,0.000000,0.014925,0.0,0.014925,0.014925,0.014925,0.000000
2,M2K,Bayview Village,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.250000,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.0,0.250000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,M2M,"Willowdale, Newtonbrook",0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0

In [67]:
#Creating a new dataframe to display the top 10 venues for each PostalCode.

top_venues_no = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Neighborhoods']
freqColumns = []
for x in np.arange(top_venues_no):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(x+1, indicators[ind]))
    except:
        freqColumns.append('{}the Most Common Venue'.format(x+1))
columns = areaColumns+freqColumns

# create a new dataframe
NY_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
NY_neighborhoods_venues_sorted['PostalCode'] = NY_grouped['PostalCode']
NY_neighborhoods_venues_sorted['Neighborhoods'] = NY_grouped['Neighborhoods']

for ind in np.arange(NY_grouped.shape[0]):
    row_categories = NY_grouped.iloc[ind, :].iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    NY_neighborhoods_venues_sorted.iloc[ind, 2:] = row_categories_sorted.index.values[0:top_venues_no]

NY_neighborhoods_venues_sorted.head(10)

,PostalCode,Neighborhoods,1the Most Common Venue,2the Most Common Venue,3the Most Common Venue,4the Most Common Venue,5the Most Common Venue,6the Most Common Venue,7the Most Common Venue,8the Most Common Venue,9the Most Common Venue,10the Most Common Venue
0,M2H,Hillcrest Village,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant
1,M2J,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Electronics Store,Chocolate Shop,Japanese Restaurant,Juice Bar,Bank,Bakery
2,M2K,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,M2M,"Willowdale, Newtonbrook",Park,Vietnamese Restaurant,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,M2N,"Willowdale, Willowdale East",Ramen Restaurant,Pizza Place,Restaurant,Café,Sushi Restaurant,Sandwich Place,Coffee Shop,Bubble Tea Shop,Pet Store,Movie Theater
5,M2P,York Mills West,Park,Convenience Store,Electronics Store,Vietnamese Restaurant,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
6,M2R,"Willowdale, Willowdale West",Coffee Shop,Butcher,Grocery Store,Pharmacy,Pizza Place,Home Service,Bank,Vietnamese Restaurant,Diner,Comfort Food Restaurant
7,M3A,Parkwoods,Park,Food & Drink Shop,BBQ Joint,Pool,Vietnamese Restaurant,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
8,M3B,Don Mills,Japanese Restaurant,Caribbean Restaurant,Café,Gym,Vietnamese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
9,M3C,Don Mills,Restaurant,Gym,Coffee Shop,Beer Store,Japanese Restaurant,Italian Restaurant,Dim Sum Restaurant,Bike Shop,Discount Store,Clothing Store


In [68]:

NY_neighborhoods_venues_sorted.shape

(23, 12)

In [75]:
df_NY_new=df_NY_new[df_NY_new.PostalCode != "M2L"]

In [32]:
df_NY_new=df_NY_dropna[df_NY_dropna.Borough == "North York"].reset_index(drop=True)

In [76]:
df_NY_new.shape

(23, 5)

## KNN clustering

In [77]:
# setting the number of clusters
kclusters = 5

NY_grouped_clustering = NY_grouped.drop(["PostalCode",  "Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

kmeans.labels_[0:10]



array([0, 0, 0, 1, 0, 4, 0, 0, 0, 0], dtype=int32)

In [78]:
# creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
NY_merge = df_NY_new.copy()

# adding clustering labels
NY_merge["Cluster Labels"] = kmeans.labels_

# merging toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
NY_merge = NY_merge.join(NY_neighborhoods_venues_sorted.drop([ "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

NY_merge.head(10) 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1the Most Common Venue,2the Most Common Venue,3the Most Common Venue,4the Most Common Venue,5the Most Common Venue,6the Most Common Venue,7the Most Common Venue,8the Most Common Venue,9the Most Common Venue,10the Most Common Venue
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Electronics Store,Chocolate Shop,Japanese Restaurant,Juice Bar,Bank,Bakery
19,M2K,North York,Bayview Village,43.786947,-79.385975,0,Chinese Restaurant,Japanese Restaurant,Café,Bank,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
21,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,1,Park,Vietnamese Restaurant,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
22,M2N,North York,"Willowdale, Willowdale East",43.770120,-79.408493,0,Ramen Restaurant,Pizza Place,Restaurant,Café,Sushi Restaurant,Sandwich Place,Coffee Shop,Bubble Tea Shop,Pet Store,Movie Theater
23,M2P,North York,York Mills West,43.752758,-79.400049,4,Park,Convenience Store,Electronics Store,Vietnamese Restaurant,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
24,M2R,North York,"Willowdale, Willowdale West",43.782736,-79.442259,0,Coffee Shop,Butcher,Grocery Store,Pharmacy,Pizza Place,Home Service,Bank,Vietnamese Restaurant,Diner,Comfort Food Restaurant
25,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,BBQ Joint,Pool,Vietnamese Restaurant,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
26,M3B,North York,Don Mills,43.745906,-79.352188,0,Japanese Restaurant,Caribbean Restaurant,Café,Gym,Vietnamese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
27,M3C,North York,Don Mills,43.725900,-79.340923,0,Restaurant,Gym,Coffee Shop,Beer Store,Japanese Restaurant,Italian Restaurant,Dim Sum Restaurant,Bike Shop,Discount Store,Clothing Store


In [79]:
NY_merge.sort_values(["Cluster Labels"], inplace=True)
NY_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1the Most Common Venue,2the Most Common Venue,3the Most Common Venue,4the Most Common Venue,5the Most Common Venue,6the Most Common Venue,7the Most Common Venue,8the Most Common Venue,9the Most Common Venue,10the Most Common Venue
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant
79,M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.713756,-79.490074,0,Park,Construction & Landscaping,Bakery,Vietnamese Restaurant,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop
72,M6B,North York,Glencairn,43.709577,-79.445073,0,Pizza Place,Asian Restaurant,Pub,Japanese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
71,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Accessories Store,Vietnamese Restaurant,Furniture / Home Store,Arts & Crafts Store,Boutique,Carpet Store,Coffee Shop,Event Space,Gift Shop
62,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0,Italian Restaurant,Sandwich Place,Coffee Shop,Butcher,Juice Bar,Café,Indian Restaurant,Comfort Food Restaurant,Breakfast Spot,Pharmacy


In [80]:
# visualize the resulting clusters

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(NY_merge['Latitude'], NY_merge['Longitude'], NY_merge['PostalCode'],
                                             NY_merge['Borough'], NY_merge['Neighborhood'], NY_merge['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [81]:
NY_merge

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1the Most Common Venue,2the Most Common Venue,3the Most Common Venue,4the Most Common Venue,5the Most Common Venue,6the Most Common Venue,7the Most Common Venue,8the Most Common Venue,9the Most Common Venue,10the Most Common Venue
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant
79,M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.713756,-79.490074,0,Park,Construction & Landscaping,Bakery,Vietnamese Restaurant,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop
72,M6B,North York,Glencairn,43.709577,-79.445073,0,Pizza Place,Asian Restaurant,Pub,Japanese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
71,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Accessories Store,Vietnamese Restaurant,Furniture / Home Store,Arts & Crafts Store,Boutique,Carpet Store,Coffee Shop,Event Space,Gift Shop
62,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0,Italian Restaurant,Sandwich Place,Coffee Shop,Butcher,Juice Bar,Café,Indian Restaurant,Comfort Food Restaurant,Breakfast Spot,Pharmacy
34,M4A,North York,Victoria Village,43.725882,-79.315572,0,French Restaurant,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Chocolate Shop,Clothing Store,Comfort Food Restaurant
33,M3N,North York,Downsview,43.761631,-79.520999,0,Grocery Store,Gym / Fitness Center,Athletics & Sports,Discount Store,Vietnamese Restaurant,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
32,M3M,North York,Downsview,43.728496,-79.495697,0,Baseball Field,Food Truck,Home Service,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
31,M3L,North York,Downsview,43.739015,-79.506944,0,Grocery Store,Park,Bank,Shopping Mall,Vietnamese Restaurant,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0,Coffee Shop,Bank,Middle Eastern Restaurant,Convenience Store,Pet Store,Pharmacy,Pizza Place,Bridal Shop,Deli / Bodega,Mobile Phone Shop


In [100]:
df_NY_venues

VenueCategory
Accessories Store              3
Airport                        1
American Restaurant            2
Art Gallery                    1
Arts & Crafts Store            2
Asian Restaurant               3
Athletics & Sports             1
BBQ Joint                      1
Bakery                         3
Bank                           7
Bar                            2
Baseball Field                 3
Beer Store                     2
Bike Shop                      1
Boutique                       2
Breakfast Spot                 1
Bridal Shop                    1
Bubble Tea Shop                1
Burger Joint                   1
Burrito Place                  1
Bus Station                    1
Butcher                        2
Café                           5
Caribbean Restaurant           2
Carpet Store                   1
Chinese Restaurant             3
Chocolate Shop                 2
Clothing Store                12
Coffee Shop                   17
Comfort Food Restaurant      

## Conclusion

As we have seen the majority of neighbourhoods are everyday service shops. A mixed bag North York has it all and certainly makes up to be a excellent neighbourhood to move into.